In [1]:
''' 신경망 라이브러리 불러오기 '''

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
''' 신경망 모델을 구성하기 '''

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 16개의 필터를 가진 4 x 4 크기의 컨볼루션 계층을 정의합니다.
W1 = tf.Variable(tf.random_normal([4, 4, 1, 16], stddev=0.01))
# 필터를 슬라이딩 할 때 한 칸씩 슬라이딩을 수행합니다.
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

# 2 x 2 크기의 풀링 계층을 정의합니다.
# 슬라이딩을 수행할 때 두 칸씩 슬라이딩을 수행합니다.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 32개의 필터를 가진 4 x 4 크기의 컨볼루션 계층을 정의합니다.
W2 = tf.Variable(tf.random_normal([4, 4, 16, 32], stddev=0.01))
# 필터를 슬라이딩 할 때 한 칸씩 슬라이딩을 수행합니다.
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

# 2 x 2 크기의 풀링 계층을 정의합니다.
# 슬라이딩을 수행할 때 두 칸씩 슬라이딩을 수행합니다.
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 2 x 2 크기의 풀링을 두 번 해서 28 / 4 = 7의 높이와 너비를 가집니다.
W3 = tf.Variable(tf.random_normal([7 * 7 * 32, 256], stddev=0.01))
# 1차원 배열로 설정합니다.
L3 = tf.reshape(L2, [-1, 7 * 7 * 32])
# 은닉층으로 256개의 뉴런으로 연결합니다.
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
# 과적합을 막기 위해 드롭아웃 기법을 사용합니다.
L3 = tf.nn.dropout(L3, keep_prob)

# 은닉층 256개의 뉴런을 입력으로 10개의 분류를 만듭니다.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

# 비용 함수를 이용해 최적화를 수행합니다.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [3]:
''' 신경망 모델 학습하기 '''

# 세션을 이용해 학습을 수행할 수 있도록 합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 배치 크기를 100으로 설정합니다.
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

# 총 10번 학습 데이터를 반복합니다.
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        total_cost += cost_val
        
    print('반복:', '%04d' % (epoch + 1), '평균 비용:', '{:.4f}'.format(total_cost / total_batch))
    
print('학습 완료!')

반복: 0001 평균 비용: 0.3457
반복: 0002 평균 비용: 0.1003
반복: 0003 평균 비용: 0.0726
반복: 0004 평균 비용: 0.0580
반복: 0005 평균 비용: 0.0472
반복: 0006 평균 비용: 0.0402
반복: 0007 평균 비용: 0.0358
반복: 0008 평균 비용: 0.0290
반복: 0009 평균 비용: 0.0258
반복: 0010 평균 비용: 0.0220
학습 완료!


In [4]:
''' 신경망 학습 결과 확인하기 '''

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                              Y: mnist.test.labels,
                                              keep_prob: 1}))

정확도: 0.9904001
